### SAKI Exercise 1 - Paul Kremer
Imports

In [52]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
import pandas as pd
import seaborn as sns

Read dataset

In [53]:
data_csv = pd.read_csv("data.csv", sep=";")
data_csv = pd.get_dummies(data_csv[['Auftragskonto', 'Buchungstag', 'Valutadatum', 'Buchungstext', 'Verwendungszweck', 'Beguenstigter/Zahlungspflichtiger', 'Kontonummer', 'label']])
data_csv.head()

#data_csv.describe(include='all')

,Auftragskonto,Buchungstag_01.02.2016,Buchungstag_01.03.2016,Buchungstag_01.04.2016,Buchungstag_01.06.2016,Buchungstag_01.07.2016,Buchungstag_02.02.2016,Buchungstag_02.03.2016,Buchungstag_02.06.2016,Buchungstag_03.02.2016,...,Kontonummer_DE90793301110000410638,Kontonummer_DE91740201000008119083,Kontonummer_DE94721500000053303830,Kontonummer_DE97500400000589011600,label_finance,label_income,label_leisure,label_living,label_private,label_standardOfLiving
0,89990201.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,89990201.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,89990201.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,89990201.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,89990201.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


Analyze Data

In [54]:
data = data_csv.to_numpy()
# sns.pairplot(data_csv)
# data[:,[1]].dtype

crop data

In [55]:
data = np.delete(data, [0, 10], axis=1) #delete first and 10th row
print(data)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


classify

In [56]:
features = data[:,:-1]
labels = data[:,-1]
# uniques_values, labels = np.unique(labels, return_inverse=True)
feat_train, feat_test, labels_train, labels_test = train_test_split(features, labels, test_size=0.5, random_state=0)
gnb = GaussianNB()
y_pred = gnb.fit(feat_train, labels_train).predict(feat_test)
print("Number of mislabeled points out of a total %d points : %d"
      % (feat_test.shape[0], (labels_test != y_pred).sum()))

Number of mislabeled points out of a total 105 points : 1


In [57]:
unique, counts = np.unique(labels, return_counts=True)
np.asarray((unique, counts)).T



array([[  0., 162.],
       [  1.,  47.]])